In [2]:
import random
import markovify
from datasets import load_dataset
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
import warnings

warnings.filterwarnings('ignore', category=RuntimeWarning)

In [3]:
def generate_markov_chain(text):
    words = text.split()
    chain = {}
    for i in range(len(words) - 1):
        current_word = words[i]
        next_word = words[i + 1]
        if current_word in chain:
            chain[current_word].append(next_word)
        else:
            chain[current_word] = [next_word]
    return chain

def generate_text(chain, length=10):
    current_word = random.choice(list(chain.keys())).strip('.')
    generated_text = [current_word]
    for _ in range(length - 1):
        if current_word in chain:
            next_word = random.choice(chain[current_word])
            next_word = next_word.strip('.')
            generated_text.append(next_word)
            current_word = next_word
        else:
            break
    generated_text[-1] += '.'
    return ' '.join(generated_text)

# Read the text from the file
with open('data/proverbe.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Generate the Markov chain
chain = generate_markov_chain(text)

# Generate a proverb using the Markov chain
generated_proverb = generate_text(chain, length=10)
print("Proverb generat:")
print(generated_proverb)

Proverb generat:
Ca musca in faina.


In [4]:
def generate_markov_chain_n_states(text, n):
    words = text.split()
    chain = {}
    for i in range(len(words) - n):
        # Extract the current state, which is a tuple of n consecutive words
        current_state = tuple(words[i:i + n])
        # Get the next word after the current state
        next_word = words[i + n]
        # Update the chain dictionary with the current state and next word
        if current_state in chain:
            chain[current_state].append(next_word)
        else:
            chain[current_state] = [next_word]
    return chain

def generate_text_n_states(chain, n, length=10):
    current_state = random.choice(list(chain.keys()))
    generated_text = list(current_state)
    for _ in range(length - n):
        if current_state in chain:
            # Randomly select the next word from the list of values associated with the current state
            next_word = random.choice(chain[current_state])
            generated_text.append(next_word)
            current_state = tuple(generated_text[-n:])
        else:
            break
    generated_text = [word.strip('.') for word in generated_text]
    if not generated_text[-1].endswith('.'):
        generated_text[-1] += '.'
    return ' '.join(generated_text)

# Read the text from the file
with open('data/proverbe.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Specify the number of states
n = 3

chain = generate_markov_chain_n_states(text, n)

# Generate a proverb using the Markov chain with n states
generated_proverb = generate_text_n_states(chain, n, length=10)
print("Proverb generat:")
print(generated_proverb)

Proverb generat:
haina si nu haina pe om Omul face haina si.


In [5]:
def load_poetry_data():
    # Load the Gutenberg Poetry Corpus
    dataset = load_dataset("biglam/gutenberg-poetry-corpus")
    if "train" in dataset:
        # Extract the poetry text
        poetry_data = dataset["train"]
        poetry_text = "\n".join(poetry_data["line"])
        return poetry_text
    else:
        print("Failed to load poetry data from the Gutenberg Poetry Corpus.")
        return None

def generate_poetry(poetry_text):
    if poetry_text:
        # Train a Markov model
        text_model = markovify.NewlineText(poetry_text, state_size=2)
        # Generate a poetry
        poetry = '\n'.join([text_model.make_sentence(tries=100) for _ in range(4)])  # 4 lines
        if not poetry.endswith((".", "!", "?", ";")):
            poetry += "."
        return poetry
    else:
        return "Failed to generate poetry stanza."


# Load poetry data
poetry_text = load_poetry_data()

# Generate poetry
generated_poetry = generate_poetry(poetry_text)

# Print generated poetry
print("Generated Poetry:")
print(generated_poetry)

Generated Poetry:
Spake the stranger brawl
Relate thee! Greater now in what she was, with pure gold.
And by degrees of heav'n
My kind and hospitable:.


In [6]:
def calculate_sentiment_textblob(text):
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity
    return sentiment_polarity

def calculate_sentiment_nltk(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores
    

# Calculate sentiment polarity of the generated poetry
sentiment_polarity = calculate_sentiment_textblob(generated_poetry)
print("Sentiment Polarity:", sentiment_polarity)

# Calculate sentiment score for the generated text
sentiment_scores = calculate_sentiment_nltk(generated_poetry)
print("Sentiment Scores:", sentiment_polarity)

Sentiment Polarity: 0.4380952380952381
Sentiment Scores: 0.4380952380952381


In [7]:
import spacy
import numpy as np
from nltk.corpus import wordnet
from nltk.corpus import wordnet_ic

# Încărcăm modelul spaCy cu embeddings
nlp = spacy.load("en_core_web_md")

def find_synonym(word):
    max_similarity = -1
    synonym = None
    word_embedding = nlp(word).vector
    
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            lemma_embedding = nlp(lemma.name()).vector
            similarity = np.dot(word_embedding, lemma_embedding) / (np.linalg.norm(word_embedding) * np.linalg.norm(lemma_embedding))
            if similarity > max_similarity:
                max_similarity = similarity
                synonym = lemma.name()
    
    return synonym

def replace_with_synonyms(poetry):
    replaced_poetry = []
    for word in poetry.split():
        if wordnet.synsets(word):
            synonym = find_synonym(word)
            replaced_poetry.append(synonym)
        else:
            replaced_poetry.append(word)
    return " ".join(replaced_poetry)


# Poezie generată
print("Generated Poetry:")
print(generated_poetry)

# Înlocuim cuvintele cu sinonime
poetry_with_synonyms = replace_with_synonyms(generated_poetry)

print("\nGenerated Poetry with Synonyms:")
print(poetry_with_synonyms)

Generated Poetry:
Spake the stranger brawl
Relate thee! Greater now in what she was, with pure gold.
And by degrees of heav'n
My kind and hospitable:.

Generated Poetry with Synonyms:
Spake the stranger brawl pertain thee! greater now in what she was, with pure gold. And by degree of heav'n My kind and hospitable:.
